# Imports

In [ ]:
!pip install timesfm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import timesfm
from sklearn.metrics import mean_absolute_error

rng = np.random.RandomState(42)

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0].


# Data

In [ ]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features_w_base.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_columns = [f'TMAX_d{n}' for n in range(1, 15)][::-1]

X_train = data.loc[data['DATE'].dt.year <= 2021, X_columns].copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022, X_columns].copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023, X_columns].copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

X_train.head()

Train: (34938, 14), (34938,)
Valid: (2920, 14), (2920,)
Test: (2920, 14), (2920,)


,TMAX_d14,TMAX_d13,TMAX_d12,TMAX_d11,TMAX_d10,TMAX_d9,TMAX_d8,TMAX_d7,TMAX_d6,TMAX_d5,TMAX_d4,TMAX_d3,TMAX_d2,TMAX_d1
0,10.6,8.3,6.1,6.1,8.3,16.7,1.7,2.8,4.4,9.4,14.4,10.0,16.1,15.0
1,8.3,6.1,6.1,8.3,16.7,1.7,2.8,4.4,9.4,14.4,10.0,16.1,15.0,15.6
2,6.1,6.1,8.3,16.7,1.7,2.8,4.4,9.4,14.4,10.0,16.1,15.0,15.6,14.4
3,6.1,8.3,16.7,1.7,2.8,4.4,9.4,14.4,10.0,16.1,15.0,15.6,14.4,17.2
4,8.3,16.7,1.7,2.8,4.4,9.4,14.4,10.0,16.1,15.0,15.6,14.4,17.2,21.1


In [ ]:
def transform_data(df):
  data_tmp = df.\
    reset_index().\
    rename(columns={'index': 'unique_id'}).\
    melt(
        id_vars=['unique_id'],
        var_name='ds',
        value_name='temperature'
    )
  data_tmp['ds'] = data_tmp['ds'].str[6:].apply(lambda x: f'2020-01-0{x}' if len(x)==1 else f'2020-01-{x}')
  data_tmp['ds'] = pd.to_datetime(data_tmp['ds'])
  data_tmp = data_tmp.\
    sort_values(['unique_id', 'ds']).\
    reset_index(drop=True)
  return data_tmp

X_train = transform_data(X_train)
X_valid = transform_data(X_valid)
X_test = transform_data(X_test)

X_train.head(20)

,unique_id,ds,temperature
0,0,2020-01-01,15.0
1,0,2020-01-02,16.1
2,0,2020-01-03,10.0
3,0,2020-01-04,14.4
4,0,2020-01-05,9.4
5,0,2020-01-06,4.4
6,0,2020-01-07,2.8
7,0,2020-01-08,1.7
8,0,2020-01-09,16.7
9,0,2020-01-10,8.3


# TimesFM v1.0

In [ ]:
tfm = timesfm.TimesFm(
    hparams=timesfm.TimesFmHparams(
        backend='gpu',
        per_core_batch_size=32,
        horizon_len=1
    ),
    checkpoint=timesfm.TimesFmCheckpoint(
        huggingface_repo_id='google/timesfm-1.0-200m-pytorch'
    )
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tfm_output_train = tfm.forecast_on_df(
    inputs=X_train,
    freq='D',
    value_name='temperature'
)
tfm_output_valid = tfm.forecast_on_df(
    inputs=X_valid,
    freq='D',
    value_name='temperature'
)
tfm_output_test = tfm.forecast_on_df(
    inputs=X_test,
    freq='D',
    value_name='temperature'
)

tfm_output_train.head()

Processing dataframe with single process.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with single process.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with single process.
Finished preprocessing dataframe.
Finished forecasting.


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,0,2020-01-15,9.069841,3.565632,5.436514,6.787530,7.898481,9.069841,10.204084,11.518014,13.257605,15.820105
1,1,2020-01-15,8.319531,2.992785,4.794437,6.079252,7.223597,8.319531,9.463639,10.831340,12.566607,15.107790
2,2,2020-01-15,8.191413,2.758181,4.536495,5.723087,7.036305,8.191413,9.423524,10.869101,12.659157,15.468853
3,3,2020-01-15,8.742289,2.820517,4.803901,6.101149,7.552015,8.742289,10.153223,11.625277,13.541452,16.452251
4,4,2020-01-15,9.765081,2.923164,5.228627,6.755295,8.265795,9.765081,11.222482,12.912066,15.019800,18.172058


In [ ]:
print(f"Train MAE: {mean_absolute_error(y_train, tfm_output_train['timesfm'])}")
print(f"Valid MAE: {mean_absolute_error(y_valid, tfm_output_valid['timesfm'])}")
print(f"Test MAE: {mean_absolute_error(y_test, tfm_output_test['timesfm'])}")

Train MAE: 4.687840957587802
Valid MAE: 5.150242509880703
Test MAE: 4.713112514737534


# TimesFM v2.0

In [ ]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=1,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
tfm_v2_output_train = tfm.forecast_on_df(
    inputs=X_train,
    freq='D',
    value_name='temperature'
)
tfm_v2_output_valid = tfm.forecast_on_df(
    inputs=X_valid,
    freq='D',
    value_name='temperature'
)
tfm_v2_output_test = tfm.forecast_on_df(
    inputs=X_test,
    freq='D',
    value_name='temperature'
)

tfm_output_train.head()

Processing dataframe with single process.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with single process.
Finished preprocessing dataframe.
Finished forecasting.
Processing dataframe with single process.
Finished preprocessing dataframe.
Finished forecasting.


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,0,2020-01-15,9.069841,3.565632,5.436514,6.787530,7.898481,9.069841,10.204084,11.518014,13.257605,15.820105
1,1,2020-01-15,8.319531,2.992785,4.794437,6.079252,7.223597,8.319531,9.463639,10.831340,12.566607,15.107790
2,2,2020-01-15,8.191413,2.758181,4.536495,5.723087,7.036305,8.191413,9.423524,10.869101,12.659157,15.468853
3,3,2020-01-15,8.742289,2.820517,4.803901,6.101149,7.552015,8.742289,10.153223,11.625277,13.541452,16.452251
4,4,2020-01-15,9.765081,2.923164,5.228627,6.755295,8.265795,9.765081,11.222482,12.912066,15.019800,18.172058


In [ ]:
print(f"Train MAE: {mean_absolute_error(y_train, tfm_v2_output_train['timesfm'])}")
print(f"Valid MAE: {mean_absolute_error(y_valid, tfm_v2_output_valid['timesfm'])}")
print(f"Test MAE: {mean_absolute_error(y_test, tfm_v2_output_test['timesfm'])}")

Train MAE: 4.623408381221061
Valid MAE: 5.069721651779462
Test MAE: 4.597148556932806
